# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [2]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [3]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x7f65184ae0d0>)

In [4]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [5]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'H7_Vaccination policy', 'H7_Flag',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDispla

In [6]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [7]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [8]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [9]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [10]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [11]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [12]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91024,Zimbabwe,NaN,Zimbabwe__nan,2020-07-27,192.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
91025,Zimbabwe,NaN,Zimbabwe__nan,2020-07-28,113.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
91026,Zimbabwe,NaN,Zimbabwe__nan,2020-07-29,62.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
91027,Zimbabwe,NaN,Zimbabwe__nan,2020-07-30,213.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0


In [13]:
# Set number of past days to use to make predictions
nb_lookback_days = 30

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d + 1]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [14]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [15]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2,
                                                    random_state=301)

In [16]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=10000,
              positive=True,
              selection='random')
# Fit model
model.fit(X_train, y_train)

Lasso(alpha=0.1, max_iter=10000, positive=True, precompute=True,
      selection='random')

In [17]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 143.93261241527352
Test MAE: 146.99429654615457


In [18]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

Day -7 NewCases 0.04388208438852578
Day -6 NewCases 0.3884012325249454
Day -5 NewCases 0.267901524283928
Day -4 NewCases 0.060639025062931166
Day -3 NewCases 0.02462600901480678
Day -2 NewCases 0.08915248411120327
Day -1 NewCases 0.1992961636420298
Day -26 C6_Stay at home requirements 5.654036241170593
Day -22 C2_Workplace closing 3.6831741362788373
Day -21 C2_Workplace closing 10.147264260411363
Intercept 27.203348427146693


In [19]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [20]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [21]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="../../../validation/data/2020-09-30_historical_ip.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 66.77601228408992
2020-08-02: 74.9569598505673
2020-08-03: 78.35858631827678
2020-08-04: 81.84713614514999
2020-08-05: 88.56976948804756
2020-08-06: 102.44459930011737
2020-08-07: 133.43901576564656
2020-08-08: 148.24921513557192
2020-08-09: 157.32817665034844
2020-08-10: 165.3678010780756
2020-08-11: 176.50491862582044
2020-08-12: 194.55036613421123
2020-08-13: 216.50298052297444
2020-08-14: 232.79329348599475
2020-08-15: 245.44676167247675
2020-08-16: 257.5603893892693
2020-08-17: 272.3478961242516
2020-08-18: 291.0531154491563
2020-08-19: 310.84743514976395
2020-08-20: 328.2390872703902
2020-08-21: 343.70195933870855
2020-08-22: 359.17768485798115
2020-08-23: 376.55530011503197
2020-08-24: 396.05065847709227
2020-08-25: 415.972237313178
2020-08-26: 434.8130475168458
2020-08-27: 447.13877619944174
2020-08-28: 474.43980797572385
2020-08-29: 498.98627191832594
2020-08-30: 521.429873214367
2020-08-31: 543.1514552196612

Predicting for Afghanistan__

2020-08-06: 2633.9456284958915
2020-08-07: 3805.5756990902555
2020-08-08: 4259.960259491119
2020-08-09: 4366.77457315297
2020-08-10: 4164.254339566802
2020-08-11: 3795.9711904509077
2020-08-12: 3736.6787297777864
2020-08-13: 4237.736937093347
2020-08-14: 4567.472682119614
2020-08-15: 4661.2365406307945
2020-08-16: 4545.428214984863
2020-08-17: 4401.397917884595
2020-08-18: 4479.716476505505
2020-08-19: 4765.6647153304475
2020-08-20: 4994.242729955988
2020-08-21: 5078.347526953055
2020-08-22: 5057.974298360148
2020-08-23: 5048.022110045073
2020-08-24: 5160.85891069595
2020-08-25: 5362.793739987219
2020-08-26: 5535.477553120077
2020-08-27: 5632.963899122409
2020-08-28: 5672.714407587701
2020-08-29: 5731.295014599495
2020-08-30: 5856.873717375756
2020-08-31: 6023.659558420579

Predicting for Burundi__nan
2020-08-01: 54.7125573579724
2020-08-02: 64.70848881823463
2020-08-03: 72.36664600031088
2020-08-04: 80.25636104883957
2020-08-05: 86.86628891766786
2020-08-06: 97.96244777519355
2020-08-

2020-08-16: 1148.3717775443317
2020-08-17: 1152.788669684983
2020-08-18: 1195.8518256836676
2020-08-19: 1264.928093442359
2020-08-20: 1308.6451752245905
2020-08-21: 1330.1067668727628
2020-08-22: 1343.1396371537535
2020-08-23: 1366.0454871860322
2020-08-24: 1410.3771652746975
2020-08-25: 1463.3076510794851
2020-08-26: 1504.923725457257
2020-08-27: 1522.8547930915622
2020-08-28: 1546.2705332450653
2020-08-29: 1578.1153845590404
2020-08-30: 1621.9184341519112
2020-08-31: 1668.803862304389

Predicting for Belarus__nan
2020-08-01: 1629.0712089507929
2020-08-02: 1843.9000966059534
2020-08-03: 1958.8226563961177
2020-08-04: 1868.9822779643578
2020-08-05: 1462.9358092361515
2020-08-06: 1179.3662723499353
2020-08-07: 1709.3329769190716
2020-08-08: 1959.9400163630417
2020-08-09: 2055.5222670693543
2020-08-10: 1954.1377274905788
2020-08-11: 1743.1416451241496
2020-08-12: 1723.6245926587164
2020-08-13: 1964.757518774081
2020-08-14: 2145.929298415059
2020-08-15: 2211.2188651282404
2020-08-16: 2153

2020-08-27: 4284.87282643789
2020-08-28: 4325.148426507885
2020-08-29: 4384.256049421119
2020-08-30: 4490.63213122354
2020-08-31: 4621.76546142342

Predicting for Brazil__Ceara
2020-08-01: 941.9678384859367
2020-08-02: 620.2380235084158
2020-08-03: 627.8514199248447
2020-08-04: 1554.4530308274873
2020-08-05: 2232.338031710934
2020-08-06: 1136.6991415136777
2020-08-07: 1086.2765720504094
2020-08-08: 969.8453351024793
2020-08-09: 1193.3476729245813
2020-08-10: 1701.66424110686
2020-08-11: 1827.4000107275485
2020-08-12: 1486.9112741465894
2020-08-13: 1357.4783914230431
2020-08-14: 1347.6896226834144
2020-08-15: 1551.3172329471115
2020-08-16: 1808.1157554493766
2020-08-17: 1849.2833478776504
2020-08-18: 1723.3991074153716
2020-08-19: 1652.8170841332424
2020-08-20: 1689.1836631179924
2020-08-21: 1836.9915351526881
2020-08-22: 1990.172188357298
2020-08-23: 2027.7104747789888
2020-08-24: 1988.6971928768583
2020-08-25: 1973.8082345929436
2020-08-26: 2028.201426997751
2020-08-27: 2139.071869062

2020-08-02: 723.8125116077754
2020-08-03: 745.8320983246235
2020-08-04: 703.033489253487
2020-08-05: 571.2702112850296
2020-08-06: 495.2911468297844
2020-08-07: 717.1425522746642
2020-08-08: 806.2901630835092
2020-08-09: 833.5939998502168
2020-08-10: 796.8829488799205
2020-08-11: 734.2389660135639
2020-08-12: 748.7015080207857
2020-08-13: 853.4666222580107
2020-08-14: 923.5415690277059
2020-08-15: 948.0866722319311
2020-08-16: 932.8398271379672
2020-08-17: 918.0005464418534
2020-08-18: 949.4726193935593
2020-08-19: 1015.6335772797612
2020-08-20: 1068.7252076116768
2020-08-21: 1093.603501666684
2020-08-22: 1108.15961233141
2020-08-23: 1124.2798542076946
2020-08-24: 1161.919582613962
2020-08-25: 1214.026636544597
2020-08-26: 1259.2357616553174
2020-08-27: 1285.0337716615206
2020-08-28: 1308.8353072000866
2020-08-29: 1337.135474774254
2020-08-30: 1377.5605608120143
2020-08-31: 1424.29240585521

Predicting for Brazil__Pernambuco
2020-08-01: 1121.674911571087
2020-08-02: 1194.1065632461757


...

2020-08-17: 2357.500585785314
2020-08-18: 2376.377882742323
2020-08-19: 2521.0165252560346
2020-08-20: 2667.244399653988
2020-08-21: 2739.2906358328055
2020-08-22: 2740.7733801293125
2020-08-23: 2734.465265236512
2020-08-24: 2786.5086939076855
2020-08-25: 2896.9045656727913
2020-08-26: 3005.9238370730563
2020-08-27: 3067.534953907102
2020-08-28: 3097.4551096330097
2020-08-29: 3129.8473737755176
2020-08-30: 3196.6106831392826
2020-08-31: 3291.6451532368073

Predicting for United States__Massachusetts
2020-08-01: 4704.592847196339
2020-08-02: 4958.446704243892
2020-08-03: 4495.97132877967
2020-08-04: 3786.017246195084
2020-08-05: 3324.957593054808
2020-08-06: 2905.1687454539497
2020-08-07: 4449.269977442885
2020-08-08: 4846.3198613676
2020-08-09: 4666.1307583902435
2020-08-10: 4258.627201284833
2020-08-11: 3942.0670722093546
2020-08-12: 4092.675587193622
2020-08-13: 4761.187367894008
2020-08-14: 5049.040202881177
2020-08-15: 4988.7875262706275
2020-08-16: 4777.0488265557715
2020-08-17: 4

2020-08-17: 5606.921604611416
2020-08-18: 5748.82776768662
2020-08-19: 6185.955767693951
2020-08-20: 6514.538356780702
2020-08-21: 6588.970176643548
2020-08-22: 6498.644026448689
2020-08-23: 6458.717684429095
2020-08-24: 6626.157683102108
2020-08-25: 6922.293368973347
2020-08-26: 7156.52361913118
2020-08-27: 7244.784075803944
2020-08-28: 7258.189400670986
2020-08-29: 7317.841495654666
2020-08-30: 7489.919096550346
2020-08-31: 7720.33150499256

Predicting for United States__North Dakota
2020-08-01: 609.2182716010454
2020-08-02: 576.0356020633069
2020-08-03: 527.7912622143177
2020-08-04: 385.03085177297703
2020-08-05: 243.1002255008792
2020-08-06: 346.2553917994527
2020-08-07: 575.4506837288573
2020-08-08: 619.0937741457114
2020-08-09: 583.718752179627
2020-08-10: 496.8701172931677
2020-08-11: 457.62545238233895
2020-08-12: 539.0758486030551
2020-08-13: 652.7644466568726
2020-08-14: 693.9823721790306
2020-08-15: 676.8671734893378
2020-08-16: 639.071477477295
2020-08-17: 640.6889864874565

2020-08-12: 2960.019868545841
2020-08-13: 3504.868968037762
2020-08-14: 3697.881262406334
2020-08-15: 3599.6016033715223
2020-08-16: 3404.381537664827
2020-08-17: 3344.3616074190286
2020-08-18: 3544.243702226334
2020-08-19: 3840.8221350268177
2020-08-20: 3976.979115110473
2020-08-21: 3949.8362190409925
2020-08-22: 3879.7738142783546
2020-08-23: 3906.39825881648
2020-08-24: 4067.500642924899
2020-08-25: 4259.049423532047
2020-08-26: 4366.620571756543
2020-08-27: 4381.726584189202
2020-08-28: 4387.54288115916
2020-08-29: 4454.739221723245
2020-08-30: 4590.601653255048
2020-08-31: 4735.014246844425

Predicting for United States__Oregon
2020-08-01: 1641.7834727995148
2020-08-02: 1620.9711078092898
2020-08-03: 1512.2777140056376
2020-08-04: 1442.1403981885044
2020-08-05: 1173.3392691538543
2020-08-06: 1046.9094575913414
2020-08-07: 1564.7415720598208
2020-08-08: 1680.6416434465616
2020-08-09: 1668.5729704233663
2020-08-10: 1577.5627224830469
2020-08-11: 1451.2525671417648
2020-08-12: 1502.5

2020-08-17: 4036.116680616441
2020-08-18: 4044.668893970621
2020-08-19: 4298.1213588549
2020-08-20: 4557.78877168436
2020-08-21: 4684.04337075734
2020-08-22: 4658.618128764217
2020-08-23: 4608.227916849475
2020-08-24: 4677.341404807707
2020-08-25: 4862.034713205116
2020-08-26: 5048.023313638382
2020-08-27: 5148.169063370713
2020-08-28: 5175.613811511104
2020-08-29: 5203.619741740178
2020-08-30: 5299.504496425561
2020-08-31: 5452.4539357661

Predicting for United States__Virgin Islands
2020-08-01: 70.17926252413955
2020-08-02: 76.8831424507554
2020-08-03: 80.96490608574086
2020-08-04: 91.57977973923666
2020-08-05: 96.70305915543213
2020-08-06: 106.59641019493569
2020-08-07: 137.22713342804715
2020-08-08: 151.76049564013067
2020-08-09: 162.66533154420915
2020-08-10: 173.1630251481725
2020-08-11: 183.5488189678993
2020-08-12: 199.9778221587879
2020-08-13: 221.32241511236992
2020-08-14: 237.8436655447128
2020-08-15: 251.75913010165164
2020-08-16: 264.86542425359426
2020-08-17: 279.31509519

2020-08-01: 70.17926252413955
2020-08-02: 76.8831424507554
2020-08-03: 80.96490608574086
2020-08-04: 91.57977973923666
2020-08-05: 96.70305915543213
2020-08-06: 106.59641019493569
2020-08-07: 137.22713342804715
2020-08-08: 151.76049564013067
2020-08-09: 162.66533154420915
2020-08-10: 173.1630251481725
2020-08-11: 183.5488189678993
2020-08-12: 199.9778221587879
2020-08-13: 221.32241511236992
2020-08-14: 237.8436655447128
2020-08-15: 251.75913010165164
2020-08-16: 264.86542425359426
2020-08-17: 279.3150951938564
2020-08-18: 297.26287709076144
2020-08-19: 316.73187443883694
2020-08-20: 334.44414431298406
2020-08-21: 350.66900409388313
2020-08-22: 356.474393965176
2020-08-23: 368.0071014847192
2020-08-24: 385.1174954533407
2020-08-25: 403.77703685940395
2020-08-26: 421.73775604574007
2020-08-27: 425.4277029720101
2020-08-28: 435.31783678492354
2020-08-29: 448.5096387265121
2020-08-30: 465.3511303458928
2020-08-31: 483.16089891698994

Predicting for Vietnam__nan
2020-08-01: 54.5219015316691

In [22]:
# Check the predictions
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
213,Aruba,NaN,2020-08-01,66.776012
214,Aruba,NaN,2020-08-02,74.956960
215,Aruba,NaN,2020-08-03,78.358586
216,Aruba,NaN,2020-08-04,81.847136
217,Aruba,NaN,2020-08-05,88.569769


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [23]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [24]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,66.77601228408992
Aruba,,2020-08-02,74.9569598505673
Aruba,,2020-08-03,78.35858631827678
Aruba,,2020-08-04,81.84713614514999
Afghanistan,,2020-08-01,157.7121761352117
Afghanistan,,2020-08-02,255.39593436154007
Afghanistan,,2020-08-03,286.8870248655943
Afghanistan,,2020-08-04,281.5833176459894
Angola,,2020-08-01,128.93381519331052


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [25]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

ModuleNotFoundError: No module named 'covid_xprize'

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [ ]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [ ]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [ ]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

In [ ]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

### Check it

In [ ]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")